In [ ]:
import collections
import random
import sys
import copy


# The following parameters correspond to "Case 1" in Fig8.

th=200
R_K=3  # R_K=lambda_{K,h}/lambda_{K,l}
R_L=1  # R_L=lambda_{L,h}/lambda_{L,l}
N=100
K=1000
W=10000
T=2000
P_K=0.2
P_L=0.8
n=5

inputFile='randomact_n'+str(n)+'_PK'+str(P_K)+'_PL'+str(P_L)+'_r'+str(R_K)+'_rL'+str(R_L)+'_th'+str(th)+'_N'+str(N)+'_K'+str(K)

dt=T/float(n)

C_K=(W-K)/float(K)
C_p=(P_K)/float(P_L)

K_o=K/float(P_K*(n*(R_K-1)*th+T))
K_n=C_o*K_o
L_n=(W-K)/float(T)
L_e=C_K*C_p*K_n
L_o=C_K*C_p*K_o
L_o=(W-K)/float(P_L*(n*(R_L-1)*th+T))
L_e=R_L*L_o


# generate the time series {K(t)}, {L(t)}

first_periodictimeDic=collections.defaultdict(int)
periodictimeDic=collections.defaultdict(int)
for ni in range(0,n):
    ti=int(ni*dt+1)
    tf=int(ti+th)
    for t in range(ti,tf+1):
        periodictimeDic[t]=t
        if ni==0:
            first_periodictimeDic[t]=t

sumz=0
tDic=collections.defaultdict(int)
z=int(round(random.expovariate(1.0/K_n),0))
tDic[1]+=z
sumz+=z
for i in range(2,T+1):
    z=random.random()
    if z<=P_K:
        if i in periodictimeDic:
            z=int(round(random.expovariate(1.0/K_n),0))
            tDic[i]+=z
            sumz+=z
        else:
            z=int(round(random.expovariate(1.0/K_o),0))
            tDic[i]+=z
            sumz+=z

tList=list(tDic.keys())
KTalphaerror=sumz-K        
if sumz-K>0:
    for i in range(int(sumz-K)):
        z=random.randint(0,len(tList)-1)
        while tList[z]<2 or tDic[tList[z]]<1:
            z=random.randint(0,len(tList)-1)
        tDic[tList[z]]-=1
        sumz-=1
if sumz-K<0:
    for i in range(int(K-sumz)):
        z=random.randint(0,len(tList)-1)
        tDic[tList[z]]+=1
        sumz+=1

tList=list(tDic.keys())
tList.sort()
temporalKDic=collections.defaultdict(int)
sumKt=0
for i in tList:
    temporalKDic[i]=tDic[i]
    sumKt+=tDic[i]
if sumKt!=K:
    print('K error!')

datatimeDic=collections.defaultdict(int)
datastartDic=collections.defaultdict(int)
temporalKList=list(temporalKDic.keys())
temporalKList.sort()
sumKt=0
for i in temporalKList:       
    datastartDic[i]+=int(temporalKDic[i])
    datatimeDic[i]+=int(temporalKDic[i])
    sumKt+=int(temporalKDic[i])
if sumKt!=K:
    print('KT error!')

sum_temporalKDic=collections.defaultdict(int)
sumKt=0
for i in range(1,T+1):
    if 1 not in temporalKDic:
        print('t=1 error!')
    if i in temporalKDic:
        sumKt+=temporalKDic[i]
    sum_temporalKDic[i]=sumKt
if sum_temporalKDic[T]!=K:
    print('resum KT error!')

LTtDic=collections.defaultdict(float)        
LTsumz=0

LTmin=2
for i in range(LTmin,T+1):
    z=random.random()
    if z<=P_L:
        if i in periodictimeDic:
            z=int(round(random.expovariate(1.0/L_e),0))
            while z>sum_temporalKDic[i-1]:
                z=int(round(random.expovariate(1.0/L_e),0))
            LTtDic[i]+=z
            LTsumz+=z
        else:
            z=int(round(random.expovariate(1.0/L_o),0))
            while z>sum_temporalKDic[i-1]:
                z=int(round(random.expovariate(1.0/L_o),0))
            LTtDic[i]+=z
            LTsumz+=z

tList=list(LTtDic.keys())
LTalphaerror=LTsumz-(W-K)        
if LTsumz-(W-K)>0:
    for i in range(int(LTsumz-(W-K))):
        z=random.randint(0,len(tList)-1)
        while LTtDic[tList[z]]<1:
            z=random.randint(0,len(tList)-1)
        LTtDic[tList[z]]-=1
        LTsumz-=1
if LTsumz-(W-K)<0:
    for i in range(int((W-K)-LTsumz)):
        z=random.randint(0,len(tList)-1)
        while tList[z]<LTmin or LTtDic[tList[z]]>=sum_temporalKDic[tList[z]-1]:
            z=random.randint(0,len(tList)-1)
        LTtDic[tList[z]]+=1
        LTsumz+=1


tList=list(LTtDic.keys())
tList.sort()
temporalLDic=collections.defaultdict(int)
sumLt=0
for i in tList:
    temporalLDic[i]=LTtDic[i]
    sumLt+=LTtDic[i]
if sumLt!=W-K:
    print('W error!')

temporalLList=list(temporalLDic.keys())
temporalLList.sort()
sumLt=0
for i in temporalLList:       
    datatimeDic[i]+=int(temporalLDic[i])
    unidatatimeDic[i]+=int(temporalLDic[i])
    sumLt+=int(temporalLDic[i])
if sumLt!=W-K:
    print('W error!')

if sumKt!=K or sumLt!=W-K:
    print('K,W: '+str(sumKt)+','+str(sumLt))
    print('degreenum: '+str(len(edgeList)))


    
# generate a temporal network

pre_activityDic=collections.defaultdict(int)
nodeDic=collections.defaultdict(int)
for node in range(N):
    nodeDic[node]=node
    z=random.random()
    y=1.0
    #y=int(float(z**(1/(1-a))))
    pre_activityDic[node]=y

activityDic=collections.defaultdict(int)
nodeList=list(nodeDic.keys())
pre_activityList=list(pre_activityDic.values())
ymax=max(pre_activityList)
neighborDic=collections.defaultdict(list)
for node in range(N):
    activityDic[node]=pre_activityDic[node]/float(ymax)
    neighborDic[node]=nodeList[:]
    neighborDic[node].remove(node)  

datastarttList=list(datastartDic.keys())
datastarttList.sort()
datatList=list(datatimeDic.keys())
datatList.sort()

checkKT=0
for i in datastarttList:
    checkKT+=datastartDic[i]
if checkKT!=K:
    print('checkKT: '+str(checkKT))

rev_activityDic=copy.deepcopy(activityDic)
rev_pre_activityDic=copy.deepcopy(pre_activityDic)
rev_neighborDic=copy.deepcopy(neighborDic)
rev_nodeList=nodeList[:]
oldedgeDic=collections.defaultdict(int)
weighttimeDic=collections.defaultdict(list)
timeDic=collections.defaultdict(list)
weightcontactList=[]
check=0
check_datastart_num=0
cum_weighttempDic=collections.defaultdict(int)
cum_linknum=0
for i in datatList:
    seq=datatimeDic[i]
    if seq>0:
        weighttempDic=collections.defaultdict(int)
        tempDic=collections.defaultdict(int)    
        if i in datastarttList:
            check_datastart_num+=datastartDic[i]
            cum_linknum+=datastartDic[i]
            while len(cum_tempDic)<cum_linknum:
                act_z=random.random()
                random.shuffle(rev_nodeList)
                for node1 in rev_nodeList:
                    if len(cum_tempDic)<cum_linknum:
                        act=rev_activityDic[node1]
                        if act_z<=act and len(rev_neighborDic[node1])!=0:
                            z=random.randint(0,len(rev_neighborDic[node1])-1)
                            node2=rev_neighborDic[node1][z]
                            rev_neighborDic[node1].remove(node2)
                            rev_neighborDic[node2].remove(node1)
                            if node1>node2:node1,node2=node2,node1
                            edgeid=node1,node2
                            if edgeid not in cum_tempDic:
                                tempDic[edgeid]=[node1,node2]
                                weighttempDic[edgeid]=[node1,node2]
                            cum_tempDic[edgeid]=[node1,node2]
                            cum_weighttempDic[edgeid]=[node1,node2]

        if seq-len(tempDic)-len(oldedgeDic)>0:
            for edgeid in oldedgeDic:
                edge=oldedgeDic[edgeid]
                weighttempDic[edgeid]=[edge[0],edge[1]]
            while len(tempDic)<seq:
                exceed_num+=1
                seq-=1

        oldedgeList=list(oldedgeDic.values())
        while len(weighttempDic)<seq:
            z=random.randint(0,len(weightcontactList)-1)
            edge=weightcontactList[z]
            weighttempDic[edge[0],edge[1]]=edge

        for edgeid in weighttempDic:
            edge=weighttempDic[edgeid]
            oldedgeDic[edgeid]=[edge[0],edge[1]]
            weighttimeDic[i].append(edge)
            weightcontactList.append(edge)

for i in range(exceed_num):
    zt=random.randint(1,T)
    oldedgeList=list(oldedgeDic.values())
    z=random.randint(0,len(oldedgeList)-1)
    edge=oldedgeList[z]
    weighttimeDic[zt].append(edge)
    timeDic[zt].append(edge)

fintimeList=list(timeDic.keys())
fintimeList.sort()
f=open('Temporal_network/'+inputFile,'w')
for i in fintimeList:
    random.shuffle(weighttimeDic[i])
    for j in range(0,len(weighttimeDic[i])):
        f.write(str(i)+'\t'+str(weighttimeDic[i][j][0])+'\t'+str(weighttimeDic[i][j][1])+'\n')
f.close()


